In [1]:
import sqlite3
import pandas as pd
import json
import h5py
import os

In [2]:
# SQLite database file
DB_FILE = "music_data.db"

### Import MusicBrainz Sample to sqlite db

In [5]:
def create_database(db_path):
    """
    Creates an SQLite database and table if they do not exist.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS music_data (
        artist_familiarity REAL,
        artist_hotttnesss REAL,
        artist_id TEXT,
        artist_latitude REAL,
        artist_location TEXT,
        artist_longitude REAL,
        artist_name TEXT,
        genre TEXT,
        release TEXT,
        song_hotttnesss REAL,
        song_id TEXT PRIMARY KEY,
        title TEXT,
        danceability REAL,
        duration REAL,
        end_of_fade_in REAL,
        energy REAL,
        song_key INTEGER,
        loudness REAL,
        mode INTEGER,
        start_of_fade_out REAL,
        tempo REAL,
        time_signature INTEGER,
        track_id TEXT,
        year INTEGER,
        artist_terms TEXT, -- JSON list
        similar_artists TEXT, -- JSON list
        bars_start TEXT, -- JSON list
        beats_start TEXT, -- JSON list
        sections_start TEXT, -- JSON list
        segments_start TEXT, -- JSON list
        tatums_start TEXT -- JSON list
    );
    """)
    
    conn.commit()
    conn.close()


In [ ]:
# # drop music_data table to clear data
# conn = sqlite3.connect(DB_FILE)
# cursor = conn.cursor()
    
# cursor.execute("""DROP TABLE music_data""")
# conn.commit()
# conn.close()

In [ ]:
# import os

# folder_path = "C:\\Users\\Gartb\\Downloads\\millionsongsubset\\MillionSongSubset"

# if os.path.exists(folder_path):
#     print("Path exists!")
# else:
#     print("Path does NOT exist!")


Path exists!


In [ ]:
def create_database(db_path):
    """
    Creates an SQLite database and table if they do not exist.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS music_data (
        artist_familiarity REAL,
        artist_hotttnesss REAL,
        artist_id TEXT,
        artist_latitude REAL,
        artist_location TEXT,
        artist_longitude REAL,
        artist_name TEXT,
        genre TEXT,
        release TEXT,
        song_hotttnesss REAL,
        song_id TEXT PRIMARY KEY,
        title TEXT,
        danceability REAL,
        duration REAL,
        end_of_fade_in REAL,
        energy REAL,
        song_key INTEGER,
        loudness REAL,
        mode INTEGER,
        start_of_fade_out REAL,
        tempo REAL,
        time_signature INTEGER,
        track_id TEXT,
        year INTEGER,
        artist_terms TEXT, -- JSON list
        similar_artists TEXT, -- JSON list
        bars_start TEXT, -- JSON list
        beats_start TEXT, -- JSON list
        sections_start TEXT, -- JSON list
        segments_start TEXT, -- JSON list
        tatums_start TEXT -- JSON list
    );
    """)
    
    conn.commit()
    conn.close()


In [20]:
def read_hdf5(file_path):
    with h5py.File(file_path, 'r') as h5:
        # Extract scalar values directly
        metadata = h5["metadata/songs"][0]  # First row
        #print(dict(zip(metadata.dtype.names, metadata)))
        analysis = h5["analysis/songs"][0]  # First row
        print(dict(zip(metadata.dtype.names, analysis)))
        year = h5["musicbrainz/songs"][0]  # First row

        # Convert list-based fields to JSON strings
        artist_terms = json.dumps([item.decode() for item in h5["metadata/artist_terms"][:]])
        similar_artists = json.dumps([item.decode() for item in h5["metadata/similar_artists"][:]])
        bars_start = json.dumps(h5["analysis/bars_start"][:].tolist())
        beats_start = json.dumps(h5["analysis/beats_start"][:].tolist())
        sections_start = json.dumps(h5["analysis/sections_start"][:].tolist())
        segments_start = json.dumps(h5["analysis/segments_start"][:].tolist())
        tatums_start = json.dumps(h5["analysis/tatums_start"][:].tolist())

        # do empty string handling on all .decode arguments
        artist_id = metadata["artist_id"].decode().strip()
        # Combine data into a dictionary (single row)
        song_data = {
            "artist_familiarity": float(metadata["artist_familiarity"]) if metadata["artist_familiarity"] else None,
            "artist_hotttnesss": float(metadata["artist_hotttnesss"]) if metadata["artist_hotttnesss"] else None,
            "artist_id": metadata["artist_id"].decode().strip() or None,
            "artist_latitude": float(metadata["artist_latitude"]) if metadata["artist_latitude"] else None,
            "artist_location": metadata["artist_location"].decode().strip() or None,
            "artist_longitude": float(metadata["artist_longitude"]) if metadata["artist_longitude"] else None,
            "artist_name": metadata["artist_name"].decode(),
            "genre": metadata["genre"].decode().strip() or None,
            "release": metadata["release"].decode().strip() or None,
            "song_hotttnesss": float(metadata["song_hotttnesss"]) if metadata["song_hotttnesss"] else None,
            "song_id": metadata["song_id"].decode().strip() or None,
            "title": metadata["title"].decode().strip() or None,
            "danceability": float(analysis["danceability"]) if "danceability" in analysis else None,
            "duration": float(analysis["duration"]) if "duration" in analysis else None,
            "end_of_fade_in": float(analysis["end_of_fade_in"]) if "end_of_fade_in" in analysis else None,
            "energy": float(analysis["energy"]) if "energy" in analysis else None,
            "song_key": int(analysis["key"]) if "key" in analysis else None,
            "loudness": float(analysis["loudness"]) if "loudness" in analysis else None,
            "mode": int(analysis["mode"]) if "mode" in analysis else None,
            "start_of_fade_out": float(analysis["start_of_fade_out"]) if "start_of_fade_out" in analysis else None,
            "tempo": float(analysis["tempo"]) if "tempo" in analysis else None,
            "time_signature": int(analysis["time_signature"]) if "time_signature" in analysis else None,
            "track_id": metadata["track_7digitalid"],
            "year": int(year["year"]) if "year" in year else None,
            "artist_terms": artist_terms,
            "similar_artists": similar_artists,
            "bars_start": bars_start,
            "beats_start": beats_start,
            "sections_start": sections_start,
            "segments_start": segments_start,
            "tatums_start": tatums_start
        }

        return song_data


In [21]:
# Function to process all HDF5 files in a folder
def process_hdf5_files(folder_path):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.h5'):
                song_data = read_hdf5(os.path.join(root, file))

                placeholders = ", ".join(["?"] * len(song_data))
                columns = ", ".join(song_data.keys())

                sql = f"INSERT OR IGNORE INTO music_data ({columns}) VALUES ({placeholders})"
                cursor.execute(sql, list(song_data.values()))
                # conn.commit()    
                # conn.close()
                # return
    conn.commit()    
    conn.close()


In [ ]:
# # ------------------------ RUN ETL PROCESS ------------------------ 
create_database()
process_hdf5_files("C:\\Users\\Gartb\\Downloads\\millionsongsubset\\MillionSongSubset")

# # ------------------------ VERIFY DATA IMPORT ------------------------

conn = sqlite3.connect("music_data.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM music_data")
count = cursor.fetchone()[0]

print(f"Number of records in music_data: {count}")

conn.close()

# # print("Data import complete!")

{'analyzer_version': np.int32(22050), 'artist_7digitalid': np.bytes_(b'a222795e07cd65b7a530f1346f520649'), 'artist_familiarity': np.float64(0.0), 'artist_hotttnesss': np.float64(218.93179), 'artist_id': np.float64(0.247), 'artist_latitude': np.float64(0.0), 'artist_location': np.int32(0), 'artist_longitude': np.int32(0), 'artist_mbid': np.int32(0), 'artist_name': np.int32(0), 'artist_playmeid': np.int32(0), 'genre': np.int32(0), 'idx_artist_terms': np.int32(0), 'idx_similar_artists': np.int32(0), 'release': np.int32(0), 'release_7digitalid': np.int32(0), 'song_hotttnesss': np.int32(0), 'song_id': np.int32(0), 'title': np.int32(0), 'track_7digitalid': np.int32(0)}
{'analyzer_version': np.int32(22050), 'artist_7digitalid': np.bytes_(b'bb9771eeef3d5b204a3c55e690f52a91'), 'artist_familiarity': np.float64(0.0), 'artist_hotttnesss': np.float64(148.03546), 'artist_id': np.float64(0.148), 'artist_latitude': np.float64(0.0), 'artist_location': np.int32(0), 'artist_longitude': np.int32(0), 'arti

KeyboardInterrupt: 

In [25]:
conn = sqlite3.connect(DB_FILE)
# cursor = conn.cursor()
sql = "SELECT * FROM music_data LIMIT 5"
res = conn.execute(sql)
data = res.fetchall()
dict(zip([description[0] for description in res.description],data[0]))

{'artist_familiarity': 0.5574602197393447,
 'artist_hotttnesss': 0.3861516314132549,
 'artist_id': 'AREJXK41187B9A4ACC',
 'artist_latitude': 46.71067,
 'artist_location': 'France',
 'artist_longitude': 1.71819,
 'artist_name': 'Raphaël',
 'genre': None,
 'release': 'Je Sais Que La Terre Est Plate (Deluxe)',
 'song_hotttnesss': 0.5479529419800353,
 'song_id': 'SOGSMXL12A81C23D88',
 'title': 'Je Sais Que La Terre Est Plate',
 'danceability': None,
 'duration': None,
 'end_of_fade_in': None,
 'energy': None,
 'song_key': None,
 'loudness': None,
 'mode': None,
 'start_of_fade_out': None,
 'tempo': None,
 'time_signature': None,
 'track_id': b' \xe6.\x00',
 'year': None,
 'artist_terms': '["chanson", "visual kei", "hip hop", "pop rock", "british pop", "jrock", "world music", "downtempo", "singer-songwriter", "latin pop", "french", "male vocalist", "alternative rock", "chill-out", "rock", "french pop", "ethnic fusion", "electronica", "pop", "acoustic", "chanson francaise", "japanese", "j po

In [16]:
conn = sqlite3.connect(DB_FILE)
#cursor = conn.cursor()
sql = "SELECT COUNT(*) FROM music_data WHERE energy IS NULL"
res = conn.execute(sql)
data = res.fetchall()
data

print(conn)

In [17]:
pd.DataFrame(data=data, columns=[description[0] for description in res.description])

,COUNT(*)
0,10000


### add genre by first correcting track_id

In [ ]:
# join a genre file
conn_csv = sqlite3.connect("C:\\Users\\Gartb\\Downloads\\msd_tagtraum_cd2c.cls\\msd_tagtraum_cd2c2.csv")
#cursor = conn.cursor()
sql = "SELECT m.track_id, m.genre, g.genre FROM music_data m JOIN genre_data g ON m.track_id = g.track_id WHERE mm.genre IS NULL; "
res = conn_csv.execute(sql)
data = res.fetchall()
data

print(conn)

# SELECT m.track_id, m.genre, g.genre
# FROM music_data m
# JOIN genre_data g
# ON m.track_id = g.track_id
# WHERE mm.genre IS NULL;



DatabaseError: file is not a database

In [4]:
#JOIN SONGS_TO_TRACKS

# Read the .txt file and ignore the 3rd column
#txt_file = "C:\\Users\\Gartb\\Downloads\\taste_profile_song_to_tracks.txt"
df = pd.read_table("https://raw.githubusercontent.com/sastafford/million-song-dataset/refs/heads/master/data/taste_profile_song_to_tracks.txt", sep='\t', names=['song_id', 'track_id', 'ignore'], usecols=[0, 1])
df['track_id'] = df['track_id'].astype(str)
#df = pd.read_table(txt_file, sep='\t', names=['song_id', 'track_id', 'ignore'], usecols=[0, 1])

# Connect to the existing SQLite database
#conn = sqlite3.connect("C:\\Users\\Gartb\\Downloads\\music_data.db")
conn = sqlite3.connect(DB_FILE)

# Write the DataFrame to the database as a new table
df.to_sql('song_to_track', conn, if_exists='replace', index=False)

# Confirm the DataFrame
print(df.head())



              song_id            track_id
0  SOAAADD12AB018A9DD  TRNCENP12903C9EF3A
1  SOAAADE12A6D4F80CC  TRSKKFK128F148B615
2  SOAAADF12A8C13DF62  TRCQMSP128F428A6F7
3  SOAAADZ12A8C1334FB  TRMDNZY128F425A532
4  SOAAAFI12A6D4F9C66  TRZEXLQ128F1491D17


In [5]:
# remove existing incorrect track_id
conn = sqlite3.connect(DB_FILE)

# Add the track_id from song_to_track to music_data
join_query = """
    UPDATE music_data
    SET track_id = NULL;
"""

conn.execute(join_query)
conn.commit()

In [6]:
#conn = sqlite3.connect("C:\\Users\\Gartb\\Downloads\\music_data.db")
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# 1. Add the track_id column to music_data
#cursor.execute("ALTER TABLE music_data ADD COLUMN track_id TEXT")

# 2. Update the track_id in music_data by joining with song_to_track
join_query = """
    UPDATE music_data
    SET track_id = (
        SELECT st.track_id
        FROM song_to_track st
        WHERE music_data.song_id = st.song_id
    )
    WHERE music_data.song_id IN (SELECT song_id FROM song_to_track);
"""

cursor.execute(join_query)
conn.commit()

print("track_id added to music_data successfully.")
conn.close()



track_id added to music_data successfully.


In [ ]:
# Function to create the SQLite database & table
def create_database_genre():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS genre_data (
       track_id PRIMARY KEY,
        genre VARCHAR(50)
    );
    """)
    
    conn.commit()
    conn.close()


In [ ]:
## John genre import
# 400k songs with genre information
# used by https://github.com/jeangelj/predict_music_taste/blob/master/Getting%20the%20Million%20Song%20Dataset%20(HDF5).ipynb

# File containing track_id and genre (web address)
GENRE_FILE_WEBADDRESS = 'http://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-MAGD-genreAssignment.cls'

def update_genres():
    # Load the genre data file into a DataFrame
    genre_df = pd.read_table(GENRE_FILE_WEBADDRESS, sep='\t', names=['track_id', 'genre'])
    
    # Connect to the SQLite database
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # Insert data into the genre_data
    genre_df.to_sql('genre_data', conn, if_exists='replace', index=False)
    
    # Update genre values in the main table using a JOIN with progress tracking
    join_query = """
        UPDATE music_data
        SET genre = (
            SELECT gd.genre
            FROM genre_data gd
            WHERE music_data.track_id = gd.track_id
        )
        WHERE music_data.track_id IN (SELECT track_id FROM music_data);
    """
    
    cursor.execute(join_query)
    
    # Commit and close connection
    conn.commit()
    conn.close()

In [8]:
update_genres()

In [10]:
conn = sqlite3.connect(DB_FILE)
#cursor = conn.cursor()
sql = "SELECT COUNT(*) FROM music_data WHERE genre IS NOT NULL"
res = conn.execute(sql)
data = res.fetchall()
data

[(1781,)]

In [ ]:
# Taylor import
#ADDING GENRE TO music_data

#conn = sqlite3.connect("C:\\Users\\Gartb\\Downloads\\music_data.db")
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Update the genre in music_data by joining with genre_data
join_query = """
    UPDATE music_data
    SET genre = (
        SELECT g.genre
        FROM genre_data g
        WHERE music_data.track_id = g.SongID
    )
    WHERE music_data.track_id IN (SELECT SongID FROM genre_data);
"""

cursor.execute(join_query)
conn.commit()

print("Genre added to music_data successfully.")
conn.close()


Genre added to music_data successfully.


In [ ]:
import sqlite3
import shutil

# Path to the current database
original_db = "C:\\Users\\Gartb\\Downloads\\music_data.db"

# Path for the backup file
backup_db = "C:\\Users\\Gartb\\Downloads\\music_data_backup.db"

# Copy the database file to create a backup
shutil.copyfile(original_db, backup_db)

print("Backup saved as music_data_backup.db")


Backup saved as music_data_backup.db


In [ ]:
#Drop Track ID and replace 

import sqlite3

conn = sqlite3.connect("C:\\Users\\Gartb\\Downloads\\music_data.db")
cursor = conn.cursor()

# Execute the SQL code
cursor.executescript("""
CREATE TABLE music_data_new AS
SELECT artist_familiarity, artist_hotttnesss, artist_id, artist_latitude, artist_location, artist_longitude,
       artist_name, genre, release, song_hotttnesss, song_id, title, danceability, duration, end_of_fade_in,
       energy, song_key, loudness, mode, start_of_fade_out, tempo, time_signature, year, artist_terms,
       similar_artists, bars_start, beats_start, sections_start, segments_start, tatums_start
FROM music_data;    

DROP TABLE music_data;

ALTER TABLE music_data_new RENAME TO music_data;
""")

conn.commit()
conn.close()

